In [ ]:
%pylab
%matplotlib inline

In [ ]:
import gocell
import scipy.ndimage as ndi

In [ ]:
pipeline = gocell.pipeline.create_default_pipeline(2)

In [ ]:
g_raw = gocell.io.imread('~/Documents/data/isbi2013/N2DH-GOWT1/02/t%02d.tif' % 0)
imshow(g_raw, 'gray')
tight_layout()

In [ ]:
g = gocell.surface.Surface.create_from_image(ndi.gaussian_filter(g_raw, 1))

mask = zeros(g_raw.shape, bool)
mask[60:190, 250:450] = True
t = 0.1
img = (g.model - t)
imshow(img * mask, 'bwr')
colorbar()
clim(-0.1 * abs(img).max(), +0.1 * abs(img).max())
xlim(200, 500)
ylim(200, 0)
tight_layout()

In [ ]:
roi = gocell.surface.Surface(g.model.shape, g.model, mask=mask)
J, result = gocell.modelfit.modelfit(g, roi, t, 2., True, bg_radius=100)

In [ ]:
smap = full(g_raw.shape, nan)
smap[roi.mask] = result.s(J.x, J.smooth_mat)
imshow(smap, 'bwr')
colorbar()
clim(-np.nanmax(smap), +np.nanmax(smap))
xlim(200, 500)
ylim(200, 0)
tight_layout()

In [ ]:
gocell.mapper.fork.DEBUG = True

In [ ]:
data, cfg = pipeline.process_image(g_raw, cfg={
        'preprocess': {
            'remove_dark_spots': True
        },
        'seeds': {
            'max_count': 100,
            'exclude_border': False
        },
        'superpixels_entropy': {
            'sigma': 9.
        },
        'compute_candidates': {
            'max_superpixel_distance': 85,
            'max_superpixel_depth': 2
        },
        'max_setpack_weights': {
            'beta': 1
        },
        'postprocess': {
            'energy_threshold': {
                'gauss_tolerance': 1
            }
        }
    });

In [ ]:
figure(figsize=(6,6))
imshow(smoothmat2[:100, :100], interpolation='none')

In [ ]:
allclose(smoothmat2, smoothmat1)

In [ ]:
figure(figsize=(10,10))
imshow(smoothmat, interpolation='none')
tight_layout()

In [ ]:
figure(figsize=(6,6))
imshow(gocell.render.render_superpixels(data, border_radius=2))
tight_layout()

In [ ]:
figure(figsize=(6,6))
imshow(gocell.render.render_activity_regions(data))
tight_layout()

In [ ]:
figure(figsize=(6,6))
imshow(gocell.render.render_model_shapes_over_image(data))
tight_layout()

In [ ]:
data, cfg = pipeline.process_image(g_raw, data=data, first_stage='max_setpack_weights',
                                   cfg=gocell.config.derive(cfg, {
        'max_setpack_weights': {
            'alpha': 0
        }
    }))

imshow(gocell.render.render_model_shapes_over_image(data))
tight_layout()

In [ ]:
imshow(gocell.render.rasterize_labels(data))
tight_layout()

In [ ]:
imshow(gocell.render.rasterize_labels(data, merge_overlap_threshold=0.8))
tight_layout()